<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/HI_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
drive.mount('/content/drive')

# Functions

In [ ]:
# Preprocess data function
def preprocess_data(matched_data):
    matched_data['distance'] = 0
    matched_data = matched_data[[str(i) == '20' for i in matched_data["depth_adj_bottom"]] ]
    matched_data['imp_c_float'] = matched_data['imp_c'].astype(float)

    # Select ID fields and numeric columns
    id_fields = matched_data[['source_dataset', 'island', 'dist_id', 'soil_column_id', 'unique_id', 'depth_top',
                              'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude', 'x_sample', 'y_sample',
                              'x_driver', 'y_driver']]
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    numeric_cols = matched_data[keep_cols].replace('', np.nan).astype(float).fillna(0)

    # Scale numeric columns
    scaler = MinMaxScaler()
    scaled_numeric_cols = scaler.fit_transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

    # Scale imp_c
    min_c = matched_data['imp_c_float'].min()
    max_c = matched_data['imp_c_float'].max()
    scaled_imp_c = (matched_data['imp_c_float'] - min_c) / (max_c - min_c)

    # Combine ID fields with scaled numeric data
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)
    numeric_df['imp_c_scaled'] = scaled_imp_c

    return numeric_df, scaler, min_c, max_c

In [ ]:
# Train model function
def train_model(preprocessed_data):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    y = preprocessed_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build and compile model
    model = Sequential([
        Dense(256, activation='relu', input_dim=X_train.shape[1]),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train model
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

In [ ]:
# Predict on reserve function
def predict_on_reserve(preprocessed_data, model, min_c, max_c):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    predictions = model.predict(X)

    inversed_predictions = predictions * (max_c - min_c) + min_c
    inversed_truth = preprocessed_data['imp_c_scaled'] * (max_c - min_c) + min_c

    df_out = preprocessed_data.copy()
    df_out['predictions'] = predictions
    df_out['inversed_predictions'] = inversed_predictions
    df_out['inversed_imp_c'] = inversed_truth

    return df_out

In [ ]:
# Evaluate model function
def evaluate_model(model, X_test, y_test, inversed_predictions, inversed_truth):
    test_loss = model.evaluate(X_test, y_test)
    r_squared = r2_score(inversed_truth, inversed_predictions)

    print("Test Loss:", test_loss)
    print("R-Squared Score:", r_squared)

    mae = mean_absolute_error(inversed_truth, inversed_predictions)
    rmse = mean_squared_error(inversed_truth, inversed_predictions, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Scatter plot of true vs predicted values
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=inversed_truth, y=inversed_predictions)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("True vs Predicted Values")
    plt.plot([min(inversed_truth), max(inversed_truth)], [min(inversed_truth), max(inversed_truth)], 'r')
    plt.show()

    # Residual plot
    residuals = inversed_truth - inversed_predictions
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel("Residuals")
    plt.title("Distribution of Residuals")
    plt.show()


In [ ]:
# Cross-validation and model training
def build_model(input_shape, output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')
    ])
    return model

In [ ]:
# Define function for rescaling
def rescale_to_minus_one_one(array):
    return 2 * (array - array.min()) / (array.max() - array.min()) - 1


In [ ]:
def train_model_a(preprocess_data):

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    X = preprocess_data[keep_cols]

    y = preprocess_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')
        ])
        return model

    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

# Load and preprocess data

In [54]:
# Load data
gdf = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/annotated_combo_imputed_SOC.gpkg')
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')
soils_csv = pd.read_csv('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')
labeled_dist = pd.read_csv('/content/drive/MyDrive/hawaii_soils/labeled_distr_annote.csv')

In [55]:
# Preprocess soils data
soils_csv = soils_csv.dropna(subset=['latitude', 'longitude'])
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

# Merge and transform GeoDataFrames
soils_gpd = pd.merge(soils_gpd, gdf[['dist_id', 'unique_id']], on='unique_id', how='inner')
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

In [56]:
# Additional preprocessing steps
drivers_gpd['x_driver'] = drivers_gpd.geometry.x
drivers_gpd['y_driver'] = drivers_gpd.geometry.y
soils_gpd['x_sample'] = soils_gpd.geometry.x
soils_gpd['y_sample'] = soils_gpd.geometry.y

In [144]:
# Reproject both GeoDataFrames to the same CRS
soils_buffered = soils_gpd.to_crs(epsg=32604)
drivers_gpd = drivers_gpd.to_crs(soils_buffered.crs)

# Apply buffer in the same CRS
soils_buffered['x_sample'] = soils_buffered.geometry.x
soils_buffered['y_sample'] = soils_buffered.geometry.y
soils_buffered.geometry = soils_buffered.geometry.buffer(1000)

# Perform spatial join
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', predicate='intersects')

In [145]:
# Preprocess data and get scaled values
numeric_df, scaler, min_c, max_c = preprocess_data(matches_within_distance)

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [153]:
#matches_within_distance contains dist_id but numeric_df doesn't -- need to get that column in the numeric_df



Index(['median ', 'median_z', 'geometry'], dtype='object')


# Prepare random forest classifier

## Random forest specific functions

In [59]:
def normal_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.normal(variable_mean, variable_sd, n)

def uniform_generator(n, min_val, max_val, min_variation, max_variation):
    variable_min = min_val + np.random.uniform(-min_variation * 2, min_variation * 2)
    variable_max = max_val + np.random.uniform(-max_variation * 2, max_variation * 2)
    return np.random.uniform(variable_min, variable_max, n)

def right_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.lognormal(variable_mean, variable_sd, n)

def left_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return -np.random.lognormal(variable_mean, variable_sd, n)

def multimodal_generator(ns, means, sds, mean_variation, sd_variation):
    samples_list = []
    for i in range(len(means)):
        variable_mean = means[i] + np.random.normal(0, mean_variation * 2)
        variable_sd = sds[i] + abs(np.random.normal(0, sd_variation * 2))
        samples_list.append(np.random.normal(variable_mean, variable_sd, ns[i]))
    return np.concatenate(samples_list)

In [60]:
def rescale_to_minus_one_one(values):
    min_value = np.min(values)
    max_value = np.max(values)
    scaled_values = 2 * ((values - min_value) / (max_value - min_value)) - 1
    return scaled_values

In [61]:
def generate_samples(n, q, generator_func, *args):
    samples = generator_func(n, *args)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)

def generate_normal_samples(n=10000, q=10, mean_value=0, sd_value=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, normal_generator, mean_value, sd_value, mean_variation, sd_variation)

def generate_uniform_samples(n=10000, q=10, min_value=-1, max_value=1, min_variation=0.5, max_variation=0.5):
    return generate_samples(n, q, uniform_generator, min_value, max_value, min_variation, max_variation)

def generate_right_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, right_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_left_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, left_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_multimodal_samples(ns=[1000, 50], q=10, means=[1, -1], sds=[1, 0.5], mean_variation=0.5, sd_variation=0.5):
    samples = multimodal_generator(ns, means, sds, mean_variation, sd_variation)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)


In [62]:
def generate_training_data(n_samples=10000):
    data_list = {}

    # Generate samples for each distribution type
    data_list['normal'] = [generate_normal_samples() for _ in range(n_samples)]
    data_list['bimodal'] = [generate_multimodal_samples() for _ in range(n_samples)]
    data_list['uniform'] = [generate_uniform_samples() for _ in range(n_samples)]
    data_list['right_tailed'] = [generate_right_tailed_samples() for _ in range(n_samples)]
    data_list['left_tailed'] = [generate_left_tailed_samples() for _ in range(n_samples)]

    # Combine all data into a single DataFrame
    combined_data = pd.DataFrame()
    for name, samples_list in data_list.items():
        df = pd.DataFrame(samples_list)
        df['label'] = name
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    # Assign column names
    combined_data.columns = [f'V{i+1}' for i in range(combined_data.shape[1] - 1)] + ['label']

    return combined_data

## Train random forest classifier

In [63]:
training_data = generate_training_data()

In [64]:
# Separate features and labels
X = training_data.drop(columns=['label'])
y = training_data['label']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, min_samples_leaf=2, random_state=123)

# Train the model
rf_model.fit(X, y_encoded)

# Print the trained model
print(rf_model)

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


# Cross validation loop

In [136]:
numeric_df

,source_dataset,island,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,water,...,srad,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.144665,...,1.000000,0.0,0.006284,0.006508,0.006031,0.004547,0.987365,0.069659,0.086235,0.678571
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.074801,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987390,0.117335,0.096284,0.607143
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.063333,...,1.000000,0.0,0.006284,0.006508,0.006029,0.004547,0.987365,0.204274,0.096608,0.535714
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.000000,...,1.000000,0.0,0.006284,0.006508,0.006029,0.004545,0.987365,0.117933,0.092552,0.357143
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.202792,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987390,0.022444,0.018537,0.214286
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.243556,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987365,0.008978,0.008602,0.428571
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.248117,...,1.000000,0.0,0.006285,0.006508,0.006029,0.004547,0.987365,0.055937,0.079594,0.892857
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.207863,...,0.999994,0.0,0.006285,0.006508,0.006031,0.004545,0.987365,0.193791,0.200189,0.571429
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.112207,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987365,0.000000,0.000000,0.000000
205,FIA,Maui,FIA_1941,FIA206,0,20.0,20,20.695859,-156.136527,0.184014,...,1.000000,0.0,0.006285,0.006508,0.006031,0.004545,0.987365,0.059986,0.057804,0.428571


In [138]:
scaled_numeric_df

,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,max,...,srad,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n
205,0.144665,0.879357,0.186983,0.090512,0.326143,0.164698,0.364498,0.140518,0.146435,0.026000,...,1.000000,0.0,0.006284,0.006508,0.006031,0.004547,0.987365,0.069659,0.086235,0.678571
205,0.074801,0.959450,0.031493,0.285322,0.395021,0.061806,0.752972,0.130900,0.474641,0.147711,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987390,0.117335,0.096284,0.607143
205,0.063333,0.794962,0.099541,0.472382,0.982489,0.221933,0.912401,0.129918,0.358541,0.249522,...,1.000000,0.0,0.006284,0.006508,0.006029,0.004547,0.987365,0.204274,0.096608,0.535714
205,0.000000,0.718066,0.151882,0.420454,1.000000,0.327676,0.767149,0.128932,0.306984,0.295690,...,1.000000,0.0,0.006284,0.006508,0.006029,0.004545,0.987365,0.117933,0.092552,0.357143
205,0.202792,0.577738,0.490919,0.033324,0.374507,0.447324,0.060788,0.271460,0.073931,0.037152,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987390,0.022444,0.018537,0.214286
205,0.243556,0.581815,0.485449,0.161745,0.345370,0.406597,0.152439,0.376196,0.188488,0.094892,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987365,0.008978,0.008602,0.428571
205,0.248117,0.856766,0.193477,0.274331,0.333298,0.117170,0.535827,0.304205,0.401484,0.114524,...,1.000000,0.0,0.006285,0.006508,0.006029,0.004547,0.987365,0.055937,0.079594,0.892857
205,0.207863,0.917953,0.046097,0.473156,0.333172,0.066342,0.961797,0.278473,0.543507,0.189365,...,0.999994,0.0,0.006285,0.006508,0.006031,0.004545,0.987365,0.193791,0.200189,0.571429
205,0.112207,0.726827,0.156122,0.662077,0.633127,0.265727,1.000000,0.268470,0.523222,0.337130,...,1.000000,0.0,0.006285,0.006510,0.006031,0.004547,0.987365,0.000000,0.000000,0.000000
205,0.184014,0.861079,0.106747,0.704197,0.325380,0.114626,0.691082,0.334521,0.552995,0.177953,...,1.000000,0.0,0.006285,0.006508,0.006031,0.004545,0.987365,0.059986,0.057804,0.428571


In [134]:
# Cross-validation: loop through all dist_ids, one at a time
# subset data to exclude unlabeled distributions 1st

data = numeric_df.dropna(subset=['dist_id'])

unique_dist_ids = data['dist_id'].unique()
np.random.shuffle(unique_dist_ids)


result_list = []

# Loop over each dist_id, leaving one out at time
for xval_id in unique_dist_ids:
    r2_list = []
    loss_list = []
    prediction_list = []
    matched_data_list = []
    model_list = []
    # Reserve data for validation
    xval_data = data[data['dist_id'] == xval_id]

    # Matched data for training
    train_data = data[data['dist_id'] != xval_id]

    reserve_data = train_data.groupby('unique_id').sample(n=1)

    for i in range(10):
        matched_data = train_data.groupby('unique_id').sample(n=1)
        matched_data = matched_data.reset_index(drop=True)

        model, test_loss, r_squared, predictions, y_test = train_model_a(matched_data)

        predictions = predict_on_reserve(reserve_data, model, min_c, max_c)

        matched_data_list.append(matched_data)
        prediction_list.append(predictions)
        model_list.append(model)
        loss_list.append(test_loss)
        r2_list.append(r_squared)

    combined_array = np.array([predictions['predictions'].values for predictions in prediction_list])
    combined_array[combined_array < 0 ] = 0

    arr_min = np.min(combined_array.flatten())
    arr_max = np.max(combined_array.flatten())

    dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
    norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    scaled_numeric_df = prediction_list[0][keep_cols]

    scaled_numeric_df['norm_dist_array'] = norm_dist_array
    scaled_numeric_df.dropna(inplace=True)

    scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

    norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

    X = scaled_numeric_df[keep_cols]
    y = norm_dist_array

    # X = scaled_numeric_df.iloc[:, :-1]
    # y = scaled_numeric_df.iloc[:, -1]

    # x = np.array(X)
    # y = np.array(norm_dist_array)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = build_model(X_train.shape[1],output_shape=10)

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)


    predictions = model.predict(xval_data[keep_cols])
    imp_c_scaled = np.sort(xval_data.groupby('unique_id').sample(n=1)['imp_c_scaled']*max_c+min_c)

    imp_c_quantiles = np.quantile(imp_c_scaled, np.linspace(0, 1,10))

    scaled_predictions = predictions*max_c+min_c
    prediction_quantiles = [np.quantile(prediction, np.linspace(0, 1,10)) for prediction in scaled_predictions]

    prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  predictions]

    # Use the trained model to make predictions
    predicted_labels_encoded = rf_model.predict(prediction_rf_ready_predictions)

    # Decode the encoded labels back to original labels
    predicted_predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)

    real_rf_ready = [rescale_to_minus_one_one(sorted(prediction)+ np.random.uniform(-0.001, 0.001, prediction.shape[0])) for prediction in  [imp_c_quantiles]]

    # Use the trained model to make predictions
    real_labels_encoded = rf_model.predict(real_rf_ready)


    # Decode the encoded labels back to original labels
    real_predicted_labels = label_encoder.inverse_transform(real_labels_encoded)

    lat_lon = [Point(xy) for xy in zip(xval_data['latitude'],xval_data['longitude'])]
    xy_sample = [Point(xy) for xy in zip(xval_data['x_sample'],xval_data['y_sample'])]
    xy_driver = [Point(xy) for xy in zip(xval_data['x_driver'],xval_data['y_driver'])]

    result_dict ={
                  "xval_id": xval_id,
                  "lat_lon": lat_lon,
                  "xy_sample": xy_sample,
                  "xy_driver": xy_driver,
                  "imp_c_scaled": imp_c_scaled,
                  "imp_c_quantiles": imp_c_quantiles,
                  "real_predicted_labels": real_predicted_labels,
                  "scaled_predictions": scaled_predictions,
                  "prediction_quantiles": prediction_quantiles,
                  "predicted_predicted_labels": predicted_predicted_labels
                }
    result_list.append(result_dict)

KeyError: ['dist_id']

In [130]:
result_list[4]['imp_c_quantiles']


array([33.5729491, 33.5729491, 33.5729491, 33.5729491, 33.5729491,
       33.5729491, 33.5729491, 33.5729491, 33.5729491, 33.5729491])

In [131]:
gdf = gpd.GeoDataFrame({'median ': [i[5]for i in result_list[4]['prediction_quantiles']],'median_z': [i[5]-result_list[4]['imp_c_quantiles'][5] for i in result_list[4]['prediction_quantiles']],'geometry': result_list[4]['xy_driver']})
gdf.set_crs(epsg = 32604, inplace=True)
gdf.to_file('test.gpkg')

In [ ]:
# Calculate average R-squared and loss
average_r_squared = np.mean(r2_list)
average_loss = np.mean(loss_list)

print(average_r_squared)
print(average_loss)

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(r2_list, bins=20, color='blue', alpha=0.7)
plt.title('R-squared Distribution')
plt.xlabel('R-squared')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(loss_list, bins=20, color='red', alpha=0.7)
plt.title('Loss Distribution')
plt.xlabel('Loss')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
dist_id_performance = {}
for i, dist_id in enumerate(unique_dist_ids):
    if dist_id not in dist_id_performance:
        dist_id_performance[dist_id] = []
    dist_id_performance[dist_id].append(r2_list[i])

# Print R-squared by dist_id
for dist_id, performances in dist_id_performance.items():
    print(f"Dist_id {dist_id} - Average R-squared: {np.mean(performances)}")